## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [208]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [209]:
# load data
large = False

if large:
    path = "/large"
else:
    path = ""

players = pd.read_csv("../data/optim-data" + path + "/0_selection_players.csv", index_col=0)
n_players = players.shape[0]
market_values = pd.read_csv("../data/optim-data" + path + "/0_selection_mv.csv", index_col=0,).drop(['player_id', 'trial'], axis=1).values
# salaries = pd.read_csv("../data/optim-data/0_selection_salary.csv", index_col=0).drop('Player', axis=1).values
n_time = market_values.shape[1]
n_trials = int(market_values.shape[0]/n_players)
market_values = market_values.reshape((n_trials, n_players, n_time))

salaries = pd.read_csv("../data/optim-data" + path + "/0_selection_salaries.csv", index_col=0)[["sal0", "sal1", "sal2", "sal3", "sal4", "sal5", "sal6", "sal7", "sal8", "sal9"]].values

other_cashflows = pd.read_excel(r"..\data\optim-data\rev-exp-projections.xlsx")
other_cashflows = other_cashflows[["Revenue", "Expense"]]

# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [210]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 37
starting_budget = 995000000/1.141
minimum_budget = 1900000000

rho = 0.019
ret = 0.0014

time = np.arange(n_time)
trials = np.arange(n_trials)

In [211]:
# Decision variables
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")
lent_out = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "lentout_%s_%s")
borrowed = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "borrowed_%s_%s")

budget = ftcp.continuous_var_list(keys=time, name = "Budget_%s", lb=0)

other_revenue = ftcp.continuous_var_list(keys=time, name = "Other Revenue_%s", lb=0)
other_expenses = ftcp.continuous_var_list(keys=time, name = "Other Expenses_%s", lb=0)

lending_rev = ftcp.continuous_var_list(keys=time, name = "Lending Revenue_%s", lb=0)
borrowing_expense = ftcp.continuous_var_list(keys=time, name = "Borrowing Expense_%s", lb=0)

borrowing_wage = ftcp.continuous_var_list(keys=time, name = "Borrowing Wage_%s", lb=0)
wages = ftcp.continuous_var_list(keys=time, name = "Wages_%s", lb=0)

team_value = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Team Value_%s_%s")
sunset_value = ftcp.continuous_var_list(keys=trials, name = "Sunset Value_%s")
selection_value = ftcp.continuous_var_list(keys=trials,name = "Selection Value_%s")
optim_value = ftcp.continuous_var(name = "Optimisation Value")

In [212]:
# Non decision variables
# Total number of players owned by the team is not the same as number of players playing in tournament
N_owned = ftcp.integer_var_list(keys = time, name = 'Number of Players owned by the team_%s', ub=30)
N_team = ftcp.integer_var_list(keys = time, name = 'Number of Players in the team_%s', lb=22, ub=25)
N_raritan = ftcp.integer_var_list(keys = time, name = 'Number of Raritan Players in the team_%s', lb=18)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)

In [213]:
# Set up Constraints for number of players
for year in time:
    for trial in trials:
        ###
        # Not used in optim, but will be used as constraint to control costs
        ###
        
        ftcp.add_constraint(team_value[year, trial] == (ftcp.sum((assigned[index, year] + borrowed[index, year]) * market_values[trial, index, year] for index in range(n_players))
                            - ftcp.sum((assigned[index, year] + 1.1*borrowed[index, year] - 0.1*lent_out[index, year]) * salaries[index, year] for index in range(n_players)))/((1 + rho)**year))
    
    ftcp.add_constraint(other_revenue[year] == other_cashflows.loc[year, "Revenue"])
    # ftcp.add_constraint(other_revenue[year] == 0)
    ftcp.add_constraint(other_expenses[year] == other_cashflows.loc[year, "Expense"])
    # ftcp.add_constraint(other_expenses[year] == 0)
        
    ftcp.add_constraint(lending_rev[year] == ftcp.sum(0.1 * lent_out[index, year] * salaries[index, year] for index in range(n_players)))
    ftcp.add_constraint(borrowing_expense[year] == ftcp.sum(0.1 * borrowed[index, year] * salaries[index, year] for index in range(n_players)))

    ftcp.add_constraint(borrowing_wage[year] == ftcp.sum(borrowed[index, year] * salaries[index, year] for index in range(n_players)))
    ftcp.add_constraint(wages[year] == ftcp.sum((assigned[index, year] - lent_out[index, year]) * salaries[index, year] for index in range(n_players)))

    previous_budget = starting_budget if year == 0 else budget[year-1]
    ftcp.add_constraint(budget[year] == previous_budget * (1 + ret)
                        + other_revenue[year] + lending_rev[year]
                        - other_expenses[year] - borrowing_wage[year] - wages[year] - borrowing_expense[year])
    
    if year > 5:
        savings_account = minimum_budget
    else:
        savings_account = year * int(minimum_budget/4)

    ftcp.add_constraint(budget[year] >= savings_account)

    ftcp.add_constraint(N_gk[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['FW'] for index, player in players.iterrows()))
    
    ftcp.add_constraint(N_raritan[year] == ftcp.sum((assigned[index, year] + borrowed[index, year] - lent_out[index, year]) * player['Raritan'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_team[year])
    ftcp.add_constraint(N_owned[year] == ftcp.sum(assigned[index, year] for index, player in players.iterrows()))
    
    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year] * (player['Age'] + year) <= max_age)
        
        ftcp.add_constraint(assigned[index, year] <= (player['Raritan']))
        ftcp.add_constraint(lent_out[index, year] <= assigned[index, year])
        ftcp.add_constraint(borrowed[index, year] <= (1 - player['Raritan']))
        
for trial in trials:        
    ftcp.add_constraint(sunset_value[trial] == ftcp.sum((assigned[index, time[-1]] + borrowed[index, time[-1]]) * market_values[trial, index, time[-1]] for index in range(n_players))/((1 + rho)**(n_time)))
    ftcp.add_constraint(selection_value[trial] == sunset_value[trial]
        + team_value[0, trial] * 1
        + team_value[1, trial] * 1
        + team_value[2, trial] * 2
        + team_value[3, trial] * 2
        + team_value[4, trial] * 3
        + team_value[5, trial] * 3
        + team_value[6, trial] * 3
        + team_value[7, trial] * 4
        + team_value[8, trial] * 4
        + team_value[9, trial] * 6
        )

ftcp.add_constraint(optim_value == ftcp.sum(selection_value))

C:\Users\yunwa\AppData\Local\Temp\ipykernel_15828\862434681.py:30: RuntimeWarning: overflow encountered in long_scalars
  savings_account = year * int(minimum_budget/4)


docplex.mp.LinearConstraint[](Optimisation Value,EQ,Selection Value_0+Selection Value_1+Selection Value_2+Selection Value_3+Selection Value_4+Selection Value_5+Selection Value_6+Selection Value_7+Selection Value_8+Selection Value_9+Selection Value_10+Selection Value_11+Selection Value_12+Selection Value_13+Selection Value_14+Selection Value_15+Selection Value_16+Selection Value_17+Selection Value_18+Selection Value_19+Selection Value_20+Selection Value_21+Selection Value_22+Selection Value_23+Selection Value_24+Selection Value_25+Selection Value_26+Selection Value_27+Selection Value_28+Selection Value_29+Selection Value_30+Selection Value_31+Selection Value_32+Selection Value_33+Selection Value_34+Selection Value_35+Selection Value_36+Selection Value_37+Selection Value_38+Selection Value_39+Selection Value_40+Selection Value_41+Selection Value_42+Selection Value_43+Selection Value_44+Selection Value_45+Selection Value_46+Selection Value_47+Selection Value_48+Selection Value_49)

In [214]:
ftcp.maximize(optim_value)
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 73401
   - binary=72660, integer=70, continuous=671
 - number of constraints: 97631
   - linear=97631
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [215]:
ftcp.solve(log_output=True, clean_before_solve=True)
ftcp.solve_status

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 95520 rows and 47259 columns.
Aggregator did 24 substitutions.
Reduced MIP has 2087 rows, 26118 columns, and 1343844 nonzeros.
Reduced MIP has 25522 binaries, 70 generals, 0 SOSs, and 0 indicators.
Presolve time = 2.25 sec. (1372.10 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 2087 rows, 26118 columns, and 1343844 nonzeros.
Reduced MIP has 25522 binaries, 86 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.61 sec. (627.56 ticks)
Probing fixed 0 vars, tightened 7 bounds.
Probing time = 0.03 sec. (20.49 ticks)
Cover probing fixed 0 vars, tightened 31 bounds.
Clique table members: 1491.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 1.20 sec. (501.95 ticks)

        Nodes            

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [216]:
ftcp.solution.export("../data/optim-data" + path + "/optim_solution.json")

In [217]:
# import docplex.mp.conflict_refiner as cr

# ftcp.solve()
# solve_status = ftcp.get_solve_status()
# if solve_status.name in ['INFEASIBLE_SOLUTION', 'INFEASIBLE_OR_UNBOUNDED_SOLUTION']:
#     cref = cr.ConflictRefiner()
#     print('show some of the constraints that can be removed to arrive at a minimal conflict')
#     cref_result = cref.refine_conflict(ftcp, display=True)  # display flag is to show the conflicts

#     cref_result.display()
#     cref_result.as_output_table()